In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# !pip install koreanize-matplotlib
import koreanize_matplotlib

%config InlineBackend.figure_format = 'retina'

## 데이터 로드

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/Apache_Parquet_logo.svg/330px-Apache_Parquet_logo.svg.png">


### Apache Parquet

* 효율적인 데이터 저장 및 검색을 위해 설계된 오픈 소스, 열 지향 데이터 파일 형식
* 복잡한 데이터를 대량으로 처리하기 위해 향상된 성능과 함께 효율적인 데이터 압축 및 인코딩 체계를 제공
* Parquet은 Java, C++, Python 등을 포함한 여러 언어를 지원

* Twitter 와 Cloudera 의 협업으로 만들어졌습니다.
* Hadoop 창시자인 더그커팅의 trevni 열 저장 형식을 개선하기 위해 설계 되었습니다.
* 열의 값은 물리적으로 인접한 메모리 위치에 저장됩니다.
* 열 단위 압축은 효율적이고 저장 공간을 절약합니다.
* 열 값이 동일한 데이터 타입이기 때문에 압축에 유리합니다.
* 특정 열 값을 가져오는 쿼리는 전체 행 데이터를 읽을 필요가 없으므로 성능이 향상됩니다.
* 각 열에 다른 인코딩 기술을 적용할 수 있습니다.


* 열 스토리지, 필요한 데이터만 읽기
* 효율적인 바이너리 패킹
* 압축 알고리즘 및 인코딩 선택
* 데이터를 파일로 분할하여 병렬 처리 가능
* 논리 유형의 범위
* 메타데이터에 저장된 통계를 통해 불필요한 청크를 건너뛸 수 있습니다.
* 디렉토리 구조를 사용한 데이터 분할


* [Apache Parquet - Wikipedia](https://en.wikipedia.org/wiki/Apache_Parquet)

* [Development update: High speed Apache Parquet in Python with Apache Arrow - Wes McKinney](https://wesmckinney.com/blog/python-parquet-update/)


<img src="https://wesmckinney.com/images/parquet_benchmarks.png">

In [4]:
df_csv = pd.read_csv("data/online_retail.csv")
df_csv.shape

(541909, 8)

In [5]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
def downcast(df_chunk):
    display(df_chunk.head())
    for col in df_chunk.columns:
        dtypes_name = df_chunk[col].dtypes.name
        if dtypes_name.startswith("float"):
            df_chunk[col] = pd.to_numeric(df_chunk[col], downcast="float")
        elif dtypes_name.startswith("int"):
            # 최솟값을 구해서 음수가 있을 때는 integer
            # 음수가 없을 때는 unsigned
            if df_chunk[col].min() < 0 :
                df_chunk[col] = pd.to_numeric(df_chunk[col], downcast="integer")
            else:
                df_chunk[col] = pd.to_numeric(df_chunk[col], downcast="unsigned")
        # 문자일 때는 category 로 변경해 줍니다.
        # 카디널리티가 높거나 텍스트 데이터에는 적합하지 않을 수 있습니다.
        elif dtypes_name.startswith("object"):
                df_chunk[col] = df_chunk[col].astype("category")
    return df_chunk

In [ ]:
df_downcast = downcast(df_csv)
df_downcast.info()

In [ ]:
df_downcast.to_parquet("data/online_retail.gzip", index=False)

In [6]:
df_gzip = pd.read_parquet("data/online_retail.gzip")
df_gzip.shape

(541909, 8)

In [8]:
df_gzip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   InvoiceNo    541909 non-null  category
 1   StockCode    541909 non-null  category
 2   Description  540455 non-null  category
 3   Quantity     541909 non-null  int32   
 4   InvoiceDate  541909 non-null  category
 5   UnitPrice    541909 non-null  float64 
 6   CustomerID   406829 non-null  float32 
 7   Country      541909 non-null  category
dtypes: category(5), float32(1), float64(1), int32(1)
memory usage: 15.1 MB
